In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_decomposition import PLSRegression

from nirs4all.operators.transformations import (
    Gaussian as GS,
    Rotate_Translate as RT,
    SavitzkyGolay as SG,
    StandardNormalVariate as SNV,
    Haar
)
from nirs4all.pipeline.config import PipelineConfig

import json

pipeline = [
    # Normalize the spectra reflectance
    MinMaxScaler(feature_range=(0.1, 0.8)),

    # Generate 1 combinations of feature augmentation
    {
        "feature_augmentation": {
            "_or_": [
                GS, SNV, SG, Haar,
            ],
            "size": [3, (1,2)],
            "count": 10
        }
    },

    # Split the dataset in train and validation
    ShuffleSplit(n_splits=3, test_size=.25),

    # Normalize the y values
    {"y_processing": MinMaxScaler()},
    {
        "model": PLSRegression,
        "model_params": {
            "n_components": {
                "_range_": [1, 60, 1],
            }
        }
    }
]

config = PipelineConfig(pipeline)
print(json.dumps(config.serializable_steps()[0], indent=4))


Configuration generates 600 configurations.


AttributeError: 'PipelineConfig' object has no attribute 'to_dict'